In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse as ssp
from sklearn.model_selection import KFold
from sklearn.datasets import dump_svmlight_file,load_svmlight_file
from sklearn.utils import resample,shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
import lightgbm as lgb
import gc

In [ ]:
seed=1024
np.random.seed(seed)
train_frq=pd.read_csv('/home/kesci/work/train_feature_freq_final.csv',nrows=50000000)
train_bm25=pd.read_csv('/home/kesci/work/train_feature_bm25_final.csv')
train_pr=pd.read_csv('/home/kesci/work/train_feature_pr_final.csv')
train_ShareR=pd.read_csv('/home/kesci/work/train_feature_ShareR_final.csv')
train_ws=pd.read_csv('/home/kesci/work/train_feature_ws_final.csv')
train_oof=pd.read_csv('/home/kesci/work/train_feature_oof_final.csv')
train_CTR=pd.read_csv('/home/kesci/work/train_feature_CTR_final.csv')
train_dflib=pd.read_csv('/home/kesci/work/train_feature_difflib_final.csv')
train_w2vdis=pd.read_csv('/home/kesci/work/train_feature_w2vdis_final.csv')

test_frq=pd.read_csv('/home/kesci/work/test_feature_freq_final.csv')
test_bm25=pd.read_csv('/home/kesci/work/test_feature_bm25_final.csv')
test_pr=pd.read_csv('/home/kesci/work/test_feature_pr_final.csv')
test_ShareR=pd.read_csv('/home/kesci/work/test_feature_ShareR_final.csv')
test_ws=pd.read_csv('/home/kesci/work/test_feature_ws_final.csv')
test_oof=pd.read_csv('/home/kesci/work/test_feature_oof_final.csv')
test_CTR=pd.read_csv('/home/kesci/work/test_feature_CTR_final.csv')
test_dflib=pd.read_csv('/home/kesci/work/test_feature_difflib_final.csv')
test_w2vdis=pd.read_csv('/home/kesci/work/test_feature_w2vdis_final.csv')

In [ ]:
train=pd.concat([
                train_frq,
                train_pr,
                train_bm25,
                train_ShareR,
                train_ws,
                train_oof,
                train_CTR,
                train_dflib,
                train_w2vdis,
                #train_worddiff,
                ],axis=1)
test=pd.concat([
                test_frq,
                test_pr,
                test_bm25,
                test_ShareR,
                test_ws,
                test_oof,
                test_CTR,
                test_dflib,
                test_w2vdis,
                #test_worddiff,
                ],axis=1)

In [ ]:
del train_frq,
del train_pr,
del train_bm25,
del train_ShareR,
del train_ws,
del train_oof,
del train_CTR,
del train_dflib,
del train_w2vdis
#del train_worddiff

del test_frq,
del test_pr,
del test_bm25,
del test_ShareR,
del test_ws,
del test_oof,
del test_CTR,
del test_dflib,
del test_w2vdis

In [ ]:
gc.collect()
n_fold = 5
kf = KFold( n_splits=n_fold, shuffle=True, random_state=2017)
y=pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=50000000)['target']
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 32,
    'learning_rate': 0.01,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'save_binary': True,
    'min_data_in_leaf': 100,
    'device':'gpu',
    'gpu_platform_id':0,
    'gpu_device_id':0
}
n = 0
gc.collect()

In [ ]:
for flod_, (index_train,index_eval) in enumerate(kf.split(train.values,y.values)):
    x_train, x_eval = train.iloc[index_train], train.iloc[index_eval]
    y_train, y_eval = y[index_train], y[index_eval]
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_eval, y_eval, reference=lgb_train)
    
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000,
                    valid_sets=[lgb_eval],
                    verbose_eval=100,
                    early_stopping_rounds=200)
    gc.collect()
    print('start predicting on test...')
    testpreds = gbm.predict(test.values, num_iteration=gbm.best_iteration)
    if n > 0:
        totalpreds = totalpreds + testpreds
    else:
        totalpreds = testpreds
        gc.collect()
    n += 1
sub=pd.DataFrame()
sub['0']=totalpreds.reshape(-1,1)/5
sub.to_csv('/home/kesci/work/lgb_final.csv',index=False)